# OPTIMIZACIÓN BAYESIANA

Con frecuencia en machine learning queremos optimizar los hiperparámetros de un modelo. Por ejemplo, en una red neuronal queremos saber el número de capas, el número de unidades por capa o la tasa de aprendizaje (learning rate) que logran un mejor comportamiento.

Este problema lo podemos representar como:  

$$ x_{m} = argmin_{x \in {X}} f(x) $$ 


Es decir, encontrar el conjunto de hiperparámetros $x_m$ que minimiza la función de coste.

El problema es que las evaluaciones de la función de coste consumen muchos recursos, ya que en cada iteración hay que elegir unos hiperparámetros, entrenar el modelo con los datos de entrenamiento y calcular la función de coste.

Se podrían usar soluciones como una buena estimación inicial, una búsqueda de cuadrículas (greed search) o una búsqueda aleatoria, pero no son eficientes.

Aquí surge la optimización bayesiana, que trata de construir uno modelo de probabilidad de la función de coste, selecciona los mejores hiperparámetros con este modelo y lo evalua con la función de coste original.

Con este enfoque, se parte de una distribución inicial de los mejores hiperparámetros y se van seleccionando hiperparámetros para evaluarlos con la función objetivo y se actualiza el modelo de probabilidad. De esta forma se va actualizando el modelo P(éxito|hiperparámetros) que mapea el conjunto de hiperparámetros con una probabilidad de éxito.

En lugar de elegir los hiperparámetros del modelo de manera aleatoria, la optimización bayesiana los selecciona de una manera informada y va mejorando la selección con las evaluaciones realizadas. Los pasos que se siguen en la optimización bayesiana, que se pueden ver en detalle en este [enlace](https://gpss.cc/gpmc17/slides/LancasterMasterclass_1.pdf), son los siguientes:

- Construir un modelo de probabilidad sustituto de la función objetivo/coste.
- Encontrar los hiperparámetros que mejor se comportan según el modelo sustituto.
- Aplicar dichos hiperparámetros y evaluarlos usando la función objetivo original.
- Actualizar el modelo sustituto con los resultados de la evaluación.
- Repetir los pasos dos a cuatro hasta que alcancemos un umbral de tiempo o de calidad.  


Con el objetivo de facilitar el uso de la optimización bayesiana en Pytorch, [Facebook](https://ai.meta.com/blog/open-sourcing-ax-and-botorch-new-ai-tools-for-adaptive-experimentation/) ha puesto a disposición de la comunidad dos potentes herramientas:

- [Ax](https://ax.dev/): Una herramienta de alto nivel para gestionar, desarrollar, desplegar y automatizar experimentos adaptativos.
- [BoTorch](https://botorch.org/): Una librería de optimización bayesiana.

Ax proporciona librerías para interaccionar con BoTorch. BoTorch facilita la optimización bayesiana a través de una interfaz modular con primitivas de optimización, modelos de probabilidad sustitutos, funciones de adquisición y optimizadores. Todo ello aprovechando las ventajas de PyTorch como los grafos computacionales dinámicos y la diferenciación automática.

El objetivo de estas librerías es facilitar a los usuarios el desarrollo, despliegue y optimización de algoritmos de machine learning.

## OPTIMIZACIÓN BAYESIANA CON PYTORCH Y AX

Un proceso de optimización bayesiana tendría los siguientes pasos (definidos a partir de este [tutorial](https://arxiv.org/abs/1807.02811)):

- Definimos un proceso gaussiano previo (prior) para la función objetivo $f$.
- Observamos $f$ en $n_0$ puntos iniciales y fijamos $n= n_0$.
- Cuando $n ≤ N$:
    - Se actualiza la distribución de probabilidad posterior de $f$ usando los datos disponibles.
    - Se selecciona $x_n$ como una maximizador de la función de adquisición sobre $x$, donde la función de adquisición se calcula usando la distribución de probabilidad actual.
    - Se evalúa $y_n=f(x_n)$.
    - Se incrementa $n$.
- Se devuelve el punto evaluado con una mayor $f(x)$ o el punto con la mayor media posterior.   

Para implementar un modelo de optimización bayesiana en PyTorch, el tipo de librería que usemos va a depender del grado de control que queramos tener sobre las diferentes fases del ciclo.

Si queremos tener un control total y gestionar nosotros el ciclo hay que utilizar directamente BoTorch. Sin embargo, si queremos centrarnos en las partes principales del modelo y no gestionar el ciclo es mejor utilizar Ax.

Ax nos ofrece tres APIs (Loop, Service, and Developer) de experimentación secuencial que se diferencian en el nivel de customización. Loop es la más sencilla de usar y Developer la más customizable.

A continuación vamos a ver un ejemplo en Colab usando la **API  Loop** y el método *optimize*, que realiza todos los pasos de la optimización bayesiana y devuelve los parámetros optimizados. Cuando llamamos a optimize tenemos que especificar, entre otros:

- Los parámetros o conjunto factible del modelo y su rango de variación.
- La función de evaluación o función objetivo a utilizar, que tendremos que definir a parte o en la propia llamada. Hemos definido la función de Rosenbrock, mínimo en (1, 1), y una función convexa con mínimo en (0.4, 0.7).
- Si es un problema de *minimización* o *maximización*.
- El nombre del objetivo y del experimento.
- El número de iteraciones del ciclo N.

El código es el siguiente:

In [ ]:
# Será necesario instalar la librería ax-platform:
# pip install ax-platform

from ax import optimize
 
def rosenbrock_function(parameterization):
  x=parameterization.get(f"x{1}")
  y=parameterization.get(f"x{2}")
  z = (1-x)**2 + 100*(y-x**2)**2
  return z
 
def convex_function(parameterization):
  x=parameterization.get(f"x{1}")
  y=parameterization.get(f"x{2}")
  z = (x-0.4)**2 + (y-0.7)**2
  return z
 
best_parameters, best_values, experiment, model = optimize(
  parameters=[
   {
   "name": "x1",
   "type": "range",
   "bounds": [-3.0, 3.0],
   },
   {
   "name": "x2",
   "type": "range",
   "bounds": [-3.0, 3.0],
   },
  ],
  experiment_name="test",
  objective_name="Rosenbrock",
  evaluation_function=rosenbrock_function,
  minimize=True,
  total_trials=30,
)

Una vez ejecutado el modelo usando las funciones de evaluación, nos devuelve los mejores parámetros encontrados.

Se observa que, con la funcion de Rosenbrock, el modelo tarda más en converger al mínimo (1,1) que con una función convexa típica.